# Counterfactual Regret Minimization

In [ ]:
from random import shuffle
from typing import List
from collections import OrderedDict
from dataclasses import dataclass

## Controle do estado

* Para a implementação do CFR é necessária a construção de um `Node` resposável por analisar o estado atual e fazer os cálculos de acordo com a estratégia atual.

* Notações importantes:
    1. $S_i$ conjunto finito de escolhas/ações do jogador $i$
    2. $A = S_1 \times \cdots \times S_n$ conjunto com todas as combinações possíveis de ações simultâneas de todos os jogadores $\therefore A = \{a_1, \cdots a_n\}$, $a$ será um perfil de ação.
    
* Logo, para um perfil $a \in A$ o 'regret' de um jogador $i$ por não ter tomado determinada ação será:
$$u(s'_i, s_{-i}) - u(a)$$

* Vetores da classe:
    1. `regret_sum` -> Soma do arrependimento de não ter tomado determinada ação no atual `info_set`
    2. `strategy` -> Probabilidades das ações que devem ser escolhidas no atual `info_set`
    3. `strategy_sum` -> Soma de todas as estratégias utilizadas no atual `info_set`

In [3]:
class Node:
    
    def __init__(self, info_set_id):
        self.info_set = info_set_id
        self.regret_sum = [0] * NUM_ACTIONS
        self.strategy = [0] * NUM_ACTIONS
        self.strategy_sum = [0] * NUM_ACTIONS
    
    
    def get_strategy(self, realization_weight):
        # compute strategy through regret-matching
        
        normalizing_sum = 0
        
        for a in range(NUM_ACTIONS):
            self.strategy[a] = self.regret_sum[a] if self.regret_sum[a] > 0 else 0
            normalizing_sum += self.strategy[a]
    
        for a in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                self.strategy[a] /= normalizing_sum
            else:
                self.strategy[a] = 1/NUM_ACTIONS
                
            self.strategy_sum[a] += realization_weight*self.strategy[a]
            
        return self.strategy
    
    
    def get_average_strategy(self):
        avg_strategy = [0] * NUM_ACTIONS
        normalizing_sum = sum(self.strategy_sum)
            
        for a in range(NUM_ACTIONS):
            if normalizing_sum > 0:
                avg_strategy[a] = self.strategy_sum[a]/normalizing_sum
            else:
                avg_strategy[a] = 1/NUM_ACTIONS
                
        return avg_strategy
    
    
    def __str__(self):
        # o Infomation Set é definido pela carta na mao do jogador e a sequencia de acoes
        return f'{self.info_set}: {self.get_average_strategy()}'

## Implementação CFR

* Notações importantes:
    1. $I$ é um information set
    2. $A(I)$ são as ações permitidas em $I$
    3. $\sigma^t_i$ estratégia do jogador $i$ em um InfoSet $I_i$ com ações $A(I_i)$ com as probabilidades de escolher $a \in A(I_i)$ no tempo $t$
    4. $\sigma^t$ a estratégia de todos os jogadores juntos em $t$
    5. $\sigma_{I \rightarrow a}$ perfil em que a ação $a$ sempre é escolhida em $I$
    6. $h$ sequência de ações
    7. $\pi^\sigma(h)$ probabilidade de se ter o histórico $h$ com a estratégia $\sigma$
    8. $\pi^\sigma(I)$ probabilidade de chegar no InfoSet $I$
    9. $\pi_{-i}^\sigma(I)$ probabilidade 'counterfactual' de alcançar $I$ considerando que o jogador atual $i$ tem probabilidade $1$ de se alcançar o InfoState
    10. $Z$ conjunto com todos os $h$ terminais
    11. $h \sqsubset z, \forall z \in Z$ é a notação utilizada para o prefixo de $z$, definindo um histórico necessariamente não terminal

- **Counterfactual Value** com histórico não terminal $h$:
$$v_i(\sigma, h) = \sum_{z\in Z, h \sqsubset z} \pi^\sigma_{-i}(h, z)\cdot\pi^\sigma(h, z)\cdot u_i(z)$$

Isto é, o valor de um InfoSet $I$ para o jogador $i$ utilizando a estratégia $\sigma$ com o histórico $h$ é somar a chance do adversário estar em $I$ com o histórico $h$, a chance do jogador $i$ atingir um estado terminal com a estratégia $h$ e a utility do estado terminal $z$.

- **Counterfactual Regret** de não ter escolhido $a$ em $I$:
$$ r(h, a) = v_i(\sigma_{I \rightarrow a}, h) - v_i(\sigma, h) $$
$$\therefore r(I, a) = \sum_{h \in I} r(h, a)$$

In [ ]:
# game definitions
PASS, BET, NUM_ACTIONS = 0, 1, 2

# info_set: str --> Node    
node_map = OrderedDict()

In [ ]:
class KuhnPoker():
    
    def __init__(self):
        self.cards = [0, 1, 2]
        self.game_util = 0
        self.node_map = OrderedDict()
        self.current_infoset = None
        self.n_plays = 0
        self.history = ""
        self.current_player = None
        self.current_opponent = None
        self.payoff_value = 0
        
    
    def shuffle_cards(self):
        shuffle(self.cards)
        return
    
    
    def get_n_plays(self, history):
        self.history = history
        self.n_plays = len(self.history)
        return self.n_plays
    
    
    def get_player_opponent_id(self):
        player_id = self.n_plays % 2
        opponent_id = 1 - player_id
        
        self.current_player = player_id
        self.current_opponent = opponent_id
        
        return (player_id, opponent_id)
    
    
    def is_terminal_state(self):
        is_terminal = False
        payoff = 0
        
        if self.n_plays > 1:
            terminal_pass = (self.history[-1] == 'p')
            double_bet = (self.history[-2:] == 'bb')
            
            player_wins_opponent = (self.cards[self.current_player] > self.cards[self.current_opponent])
            
            if terminal_pass:
                is_terminal = True
                
                if self.history == 'pp':
                    payoff = 1 if player_wins_opponent else -1
                else:
                    payoff = 1
                    
            elif double_bet:
                payoff = 2 if player_wins_opponent else -2
            
        return (is_terminal, payoff)
    
    
    def set_current_node(self, info_set):
        node = self.node_map.get(info_set)
        
        if node is None:
            self.node_map[info_set] = Node(info_set)

        return
    
    
    def get_state_probability(self, probability_list):
        return probability_list[self.current_player]
    
    
    def get_current_strategy(self, state_reach_probability, info_set):
        strategy = self.node_map[info_set].get_strategy(state_reach_probability)
        return strategy
    
    
    def get_possible_actions(self):
        return [0, 1]
    
    
    def take_action(self, action):
        action = ('p' if action == 0 else 'b')
        return self.history + action
    
    def update_node_regret(self, action, regret, state_probability):
        self.node_map[self.current_infoset].regret_sum[action] += state_probability * regret
        return
        
        
    def get_player_hand(self):
        return self.cards[self.current_player]

In [4]:
def ChanceSampling_CFR(game, history, p0, p1):
    n_plays = game.get_n_plays(history)
    player, opponent = game.get_player_opponent_id()
    
    is_terminal, payoff = game.is_terminal_state()
    
    if is_terminal:
        return payoff
    
    player_hand = game.get_player_hand()
    info_set = f"{player_hand}{history}"
    
    game.set_current_node(info_set)
        
    # recursively call CFR with +action +history for each action
    state_probability = game.get_state_probability([p0, p1])
    strategy = game.get_current_strategy(state_probability, info_set)
    
    util = [0] * NUM_ACTIONS
    node_util = 0
    
    for action in game.get_possible_actions():
        next_history = game.take_action(action)
        
        util[action] = -ChanceSampling_CFR(game, next_history, p0*strategy[action], p1)\
                       if player == 0 else \
                       -ChanceSampling_CFR(game, next_history, p0, p1*strategy[action])
        
        node_util += strategy[action]*util[action]
        
    # accumulate counterfactual regret for each action
    for action in game.get_possible_actions():
        regret = util[action] - node_util
        game.update_node_regret(state_probability, regret)
        
    return node_util

In [5]:
def train(iterations):
    cards = [1, 2, 3]
    util = 0
    
    for i in range(iterations):
        shuffle(cards)
        util += ChanceSampling_CFR(cards, "", 1, 1)
        
    print(f'Avg game value: {util/iterations}')
    
    for node in node_map.values():
        print(node)

In [7]:
def train(iterations):
    kuhn_poker = KuhnPoker()
    
    for i in range(iterations):
        kuhn_poker.shuffle_cards()
        kuhn_poker.game_util += ChanceSampling_CFR(kuhn_poker, "", 1, 1)
        
    print(f'Avg game value: {kuhn_poker.game_util/iterations}')
    
    for node in kuhn_poker.node_map.values():
        print(node)

Avg game value: 0.000215
1: [0.9999806966878446, 1.93033121553279e-05]
2p: [0.9999948100169638, 5.189983036169733e-06]
1pb: [0.9999992590429808, 7.409570191613568e-07]
2b: [3.726270568374154e-05, 0.9999627372943162]
3: [1.4879756686218666e-06, 0.9999985120243314]
1p: [0.9999910842979628, 8.915702037237915e-06]
3pb: [0.5, 0.5]
1b: [0.9999985140496604, 1.4859503395396525e-06]
2: [0.49998662962577806, 0.5000133703742219]
2pb: [2.525582666777594e-06, 0.9999974744173333]
3p: [1.4866484106241842e-06, 0.9999985133515894]
3b: [1.4866484106241842e-06, 0.9999985133515894]
